## HW3
Geoffrey Woollard

My code lives in the repo https://github.com/geoffwoollard/prob_prog

## Acknowledgments
I acknowledge helpful discussions with Justice Sefas, Masoud Mokhatari, Dylan Green, and Jordan Lovrod, and many other classmates.

I gratefully acknowledge helpful code snippets from Masoud Mokhatari, Mohamad Amin Mohamadi, and Dylan Green, in particular during the implementation of Hamiltonian Monte Carlo.

# Code snippets

In [2]:
from dill.source import getsource, getsourcelines

## Importance sampling
* I modified the evaluator from hw2 to also return $\sigma$, which gets accumulated from the `log_prob` of evaluating observes

In [3]:
for line_number, function_line in enumerate(getsourcelines(evaluate)[0]):
    print(line_number, function_line,end='')

0 def evaluate(e,sigma=0,local_env={},defn_d={},do_log=False,logger_string=''):
1     # TODO: get local_env to evaluate values to tensors, not regular floats
2     # remember to return evaluate (recursive)
3     # everytime we call evaluate, we have to use local_env, otherwise it gets overwritten with the default {}
4     # if do_log: logger.info('logger_string {}'.format(logger_string))
5     if do_log: logger.info('ls {}'.format(logger_string))
6     if do_log: logger.info('e {}, local_env {}, sigma {}'.format(e, local_env, sigma))
7 
8     # get first expression out of list or list of one
9     if not isinstance(e,list) or len(e) == 1:
10         if isinstance(e,list):
11             e = e[0]
12         if isinstance(e,bool):
13             if do_log: logger.info('match case number: e {}, sigma {}'.format(e, sigma))
14             return torch.tensor(e), sigma
15         if isinstance(e, number):
16             if do_log: logger.info('match case number: e {}, sigma {}'.format(e, sig

I also wrote my own score function. It handles boolean cases by converting them to 

In [4]:
from evaluation_based_sampling import score
for line_number, function_line in enumerate(getsourcelines(score)[0]):
    print(line_number, function_line,end='')

0 def score(distribution,c):
1     """Score pytorch distributions with .log_prob, but in a robust way for the type of c
2     """
3     if isinstance(c,bool) or c.type() in ['torch.BoolTensor', 'torch.LongTensor']:
4         log_w = distribution.log_prob(c.double())
5     else:
6         log_w = distribution.log_prob(c)
7     return log_w


I added a few more distributions and boolean operation primitives, for the problems in this assignment

In [6]:
from primitives import distributions_d
for key in distributions_d.keys() :
    print(key,':')
    for line_number, function_line in enumerate(getsourcelines(distributions_d[key])[0]):
        print(line_number, function_line,end='')
    print()

normal :
0 def normal(mean_std):
1     return two_arg_op_primitive(torch.distributions.Normal,mean_std)

beta :
0 def beta(alpha_beta):
1     return two_arg_op_primitive(torch.distributions.Beta,alpha_beta)

exponential :
0 def exponential(lam):
1     return one_arg_op_primitive(torch.distributions.Exponential,lam)

uniform :
0 def uniform(low_hi):
1     return two_arg_op_primitive(torch.distributions.Uniform,low_hi)

discrete :
0 def discrete(prob_vector):
1     return one_arg_op_primitive(torch.distributions.Categorical,prob_vector)

flip :
0 def flip(prob):
1     return one_arg_op_primitive(torch.distributions.bernoulli.Bernoulli,prob)

dirichlet :
0 def dirichlet(concentration):
1     return one_arg_op_primitive(torch.distributions.dirichlet.Dirichlet,concentration)

gamma :
0 def gamma(concentration_rate):
1     return two_arg_op_primitive(torch.distributions.gamma.Gamma,concentration_rate)



In [7]:
from primitives import primitives_d
for key in ['and','or','>','<','>=','<=','='] :
    print(key,':')
    for line_number, function_line in enumerate(getsourcelines(primitives_d[key])[0]):
        print(line_number, function_line,end='')
    print()

and :
0 def and_primitive(arg1_arg2):
1     return two_arg_op_primitive(torch.logical_and,arg1_arg2)  

or :
0 def or_primitive(arg1_arg2):
1     return two_arg_op_primitive(torch.logical_or,arg1_arg2)  

> :
0 def gt_primitive(consequent_alternative):
1     return two_arg_op_primitive(torch.gt,consequent_alternative)

< :
0 def lt_primitive(consequent_alternative):
1     return two_arg_op_primitive(torch.lt,consequent_alternative)

>= :
0 def ge_primitive(consequent_alternative):
1     return two_arg_op_primitive(torch.ge,consequent_alternative)

<= :
0 def le_primitive(consequent_alternative):
1     return two_arg_op_primitive(torch.le,consequent_alternative)

= :
0 def eq_primitive(consequent_alternative):
1     return two_arg_op_primitive(torch.eq,consequent_alternative)



## MH within Gibbs

I implented Metropolis-Hastings within Gibbs in the following manner
* parse the graph in `mh_gibbs_wrapper`
* topologically sort the graph vertices 
* sample from the joint (ie prior) to initialize all values of the graph
* cycle through the graph with `gibbs_step` 
* accept an update at a specific vertex with `accept`
* collect each state after a Gibbs update (all the vertices) 
* return all the fully specified graphs in `gibbs` for `num_steps`
* finally, I evaluate return value (the meaning of the program) for all the sampled graphs with `evaluate_program_return_from_samples_whole_graph`

In [207]:
from mh_gibbs import mh_gibbs_wrapper,gibbs_step,accept,gibbs,evaluate_program_return_from_samples_whole_graph
list_of_programs = [mh_gibbs_wrapper,gibbs_step,accept,gibbs,evaluate_program_return_from_samples_whole_graph]

for program in list_of_programs:
    for line_number, function_line in enumerate(getsourcelines(program)[0]):
        print(line_number, function_line,end='')
    print()

0 def mh_gibbs_wrapper(graph,num_steps,do_log=False):
1     G = graph[1]
2     verteces = G['V']
3     A = G['A']
4     P = G['P']
5     X = set(verteces) - set(G['Y'].keys())
6     Y = G['Y']
7     Y = {key:evaluate([Y[key]], do_log=do_log)[0] for key in Y.keys()}
8 
9 
10     verteces_topsorted = sample_from_joint_precompute(graph)
11     _, local_env = sample_from_joint(graph,verteces_topsorted=verteces_topsorted)
12     local_env = {**local_env,**Y}
13     local_env_list0 = [local_env]
14 
15     local_env_list = gibbs(num_steps,local_env,P,A,X,do_log=do_log)
16 
17     local_env_list = local_env_list0 + local_env_list
18 
19     return_list, samples_whole_graph = evaluate_program_return_from_samples_whole_graph(graph,local_env_list)
20 
21     
22 
23     return local_env_list

0 def gibbs_step(local_env,P,A,X_sample_vertices,do_log):
1     for vertex in X_sample_vertices:
2         link_function = P[vertex]
3         e = link_function[1]
4 
5         distribution, sigma = evaluat

## Hamiltonian Monte Carlo

I implemented HMC in the following way
* I parse the graph for the link functions P, and the samples X and observes Y
* I turn on autodiff on the torch.tensor(float): `turn_on_autodiff`
* I run HMC algorithm 20 from the textbook: `hmc_algo20`
  * inside I use the leapfrog algorithm 19 from the textbook: `leapfrog`
    * this relies on computing the gradient of the potential energy with respect to the values of X: `grad_U`. There are important impelentation details with pytorch autodiff, avoiding gradient accumulation
    * I also have to add Xt (a dict) and Rt (a vector) with a helper function `add_dict_to_tensor`. I use `X_vertex_names_to_idx_d` to keep track of what key in `X` corresponds to what index of `R`. This is important if the keys change order and `M` is different for different values of `X` (i.e. not proportional to the identity matrix)
  * I compute the kinetic and potential energy and the hamiltonian: `compute_K`, `compute_U` (just negative of  `compute_log_joint_prob`) and `compute_H`
* After I collect samples from the whole graph, I evaluate the return function on each graph.
   

In [208]:
from hmc import hmc_wrapper,turn_on_autodiff,hmc_algo20,leapfrog,grad_U,add_dict_to_tensor,compute_K,compute_log_joint_prob,compute_U,compute_H

list_of_programs = [hmc_wrapper,turn_on_autodiff,hmc_algo20,leapfrog,grad_U,add_dict_to_tensor,compute_K,compute_log_joint_prob,compute_U,compute_H]

for program in list_of_programs:
    for line_number, function_line in enumerate(getsourcelines(program)[0]):
        print(line_number, function_line,end='')
    print()

0 def hmc_wrapper(graph,num_samples,T=10,epsilon=0.1,M=tensor(1.)):
1     #set up X, Y list of verteces
2     G = graph[1]
3     verteces = ['V']
4     Y = G['Y']
5     P = G['P']
6     
7     # evaluate to constants
8     Y = {key:evaluate([value])[0] for key,value in Y.items()}
9     
10     #X = set(vertices) - set(Y.keys())
11     #X = sample_from_joint(graph)
12     _, X0 = sample_from_joint(graph) # does not include observes
13     
14     # initialize in dict
15     X_vertex_names_to_idx_d = {key:idx for idx,key in enumerate(X0.keys())}
16 
17     # set up autograd on tensors
18     turn_on_autodiff(X0)
19     turn_on_autodiff(Y) # TODO: why do we need this?
20 
21     
22     # run HMC algorithm 20 from book
23         # inside use leapfrog algorithm 19 from book
24         # include kinetic and potential energy functions
25         # MC acceptance criteria
26     samples_whole_graph = hmc_algo20(X0,num_samples,T,epsilon,M,Y,P,X_vertex_names_to_idx_d)
27     
28     # evaluate 